# Homework 2 <br>


## Problem 2: Borrow code from Homework 1

In [24]:
class Node():
    def __init__(self, name, heuristic):
        self.cost = {}   
        self.children = []
        self.name = name  
        self.heuristic = heuristic 
    
    def add_children(self, node, cost):
        """
        Add the children nodes with the corresponding cost of connection
        """
        self.cost[node.name] = {}
        self.cost[node.name]['node'] = node # cost is the path cost
        self.children.append(node)
        self.cost[node.name]['cost'] = cost # cost is the path cost   

    def clear_children(self):
        self.cost = {}
    
    def sort_children(self):
        """
        sort the children alphabetically. 
        e.g. [D,A,B] -> [D,B,A] 
        """
        self.children = []
        for i in sorted(self.cost.keys(), reverse=True):
            self.children.append(self.cost[i]['node'])

    def is_terminal_node(self):
        return not self.cost

    def pop(self):
        if len(self.children):
            return self.children.pop()
        else:
            return False

class Graph():
    def __init__(self):
        # create nodes for examples in appendix and question 3
        self.S = Node('S',10)
        self.A = Node('A',float('inf'))
        self.B = Node('B',7)
        self.D = Node('D',10)
        self.E = Node('E',3)
        self.G = Node('G',0)
        self.T = Node('T',4)
        self.F = Node('F',float('inf'))
        self.J = Node('J',6)


    def create_graph_p1(self):
        """
        Create graph for problem 1 in Homework 2
        """
        # create list containing nodes needed for this graph
        node_list = [self.S, self.A, self.B, self.D, self.E, self.G, self.T, self.F, self.J]

        # clear the existing children in the nodes if any
        for node in node_list:
            node.clear_children()

        # add children node and the cost of connection
        self.S.add_children(self.D,2)
        self.S.add_children(self.B,4)
        self.D.add_children(self.A,1)
        self.D.add_children(self.E,5)
        self.B.add_children(self.T,2)
        self.B.add_children(self.J,1)
        self.A.add_children(self.F,3)
        self.T.add_children(self.G,3)
        self.J.add_children(self.G,6)
        self.E.add_children(self.J,1)

        # sort the children alphabetically
        for node in node_list:
            node.sort_children()


def Uniform_Cost_Tree_Search(start_node):
    frontier = [[start_node]]
    paths_cost = [0]
    expansion_order = []

    while len(frontier):  # while frontier is not empty
        main_node = frontier[-1][-1] # frontier[-1] gives the last path in the list, frontier[-1][-1] gives the last node in that path
        expansion_order.append(main_node) # add it to the expansion order list

        # check if the node is goal or not
        if main_node.name == 'G':
            return frontier[-1], expansion_order  # if true, return the last path in frontier and expansion order

        # get the list of successors of main node
        successors = main_node.children

        # iterate through each successor
        for successor in successors:
            tmplist = frontier[-1].copy()     
            tmplist.append(successor)    # add the successor to the path
            frontier.insert(0,tmplist)  # insert to the front of frontier list
            paths_cost.insert(0,paths_cost[-1]+main_node.cost[successor.name]['cost']) # update the path cost


        frontier.pop()  # remove the old path
        paths_cost.pop() # remove the old cost

        # min_ind = paths_cost.index(min(paths_cost))
        min_ind_list = [i for i in range(len(paths_cost)) if paths_cost[i] == min(paths_cost)] # find the indices of lowest cost path
        if (len(min_ind_list) > 1): # if these is more than one lowest cost path
            # follow the alphabetical expansion order
            lower_letter = frontier[min_ind_list[0]][-1].name  
            for i in min_ind_list[1:]:
                if frontier[i][-1].name < lower_letter: 
                    min_ind = i  
                    lower_letter = frontier[i][-1].name
        else: 
            min_ind = min_ind_list[0]

        # brind the lowest cost path to end of the list. Update the paths_cost accordingly
        frontier.append(frontier[min_ind])
        frontier.pop(min_ind)
        paths_cost.append(paths_cost[min_ind])
        paths_cost.pop(min_ind)

    return frontier, expansion_order 

def Uniform_Cost_Graph_Search(start_node):
    frontier = [[start_node]]
    paths_cost = [0]
    expansion_order = []
    explored = [start_node]

    while len(frontier):  # while frontier is not empty
        main_node = frontier[-1][-1] # frontier[-1] gives the last path in the list, frontier[-1][-1] gives the last node in that path
        expansion_order.append(main_node) # add it to the expansion order list

        # check if the node is goal or not
        if main_node.name == 'G':
            return frontier[-1], expansion_order, # if true, return the last path in frontier and expansion order

        # get the list of successors of main node
        successors = main_node.children
        # iterate through each successor
        for successor in successors:
            if successor in expansion_order: continue # if the main_node is already expanded, then skip it
            tmplist = frontier[-1].copy()     
            tmplist.append(successor)    # add the successor to the path
            frontier.insert(0,tmplist)  # insert to the front of frontier list
            paths_cost.insert(0,paths_cost[-1]+main_node.cost[successor.name]['cost']) # update the path cost

            if successor in explored: # if the main_node is already explored
                for ind, path in enumerate(frontier[1:]):  # find the node in the frontier
                    if path[-1] == successor:       
                        if paths_cost[ind+1] > paths_cost[0]:  # if the node in frontier has higher path cost,
                            frontier.pop(ind+1)                  # remove it 
                            paths_cost.pop(ind+1)
                        else:
                            frontier.pop(0)       # if the node in frontier has lower path cost,
                            paths_cost.pop(0)     # remove the successor
                        break  # jumpt out of the for loop
            else:
                explored.append(successor) # add to the explored list


        frontier.pop()  # remove the old path
        paths_cost.pop() # remove the old cost

        # min_ind = paths_cost.index(min(paths_cost))
        min_ind_list = [i for i in range(len(paths_cost)) if paths_cost[i] == min(paths_cost)] # find the indices of lowest cost path
        if (len(min_ind_list) > 1): # if these is more than one lowest cost path
            # follow the alphabetical expansion order
            lower_letter = frontier[min_ind_list[0]][-1].name  
            for i in min_ind_list[1:]:
                if frontier[i][-1].name < lower_letter: 
                    min_ind = i  
                    lower_letter = frontier[i][-1].name
        else: 
            min_ind = min_ind_list[0]

        # brind the lowest cost path to end of the list. Update the paths_cost accordingly
        frontier.append(frontier[min_ind])
        frontier.pop(min_ind)
        paths_cost.append(paths_cost[min_ind])
        paths_cost.pop(min_ind)

    return frontier, expansion_order 

In [25]:
# execute the Uniform Cost graph search 
graph = Graph()
graph.create_graph_p1()

p, e = Uniform_Cost_Graph_Search(graph.S)
print("The path returned is ", end='')
for node in p:
    print(node.name, end = ' ')
print('\n')
print("The node expansion order is ", end='')
for node in e:
    print(node.name, end = ' ')

The path returned is S B T G 

The node expansion order is S D A B J F T E G 

In [26]:
# execute the Uniform Cost tree search 
graph = Graph()
graph.create_graph_p1()

p, e = Uniform_Cost_Tree_Search(graph.S)
print("The path returned is ", end='')
for node in p:
    print(node.name, end = ' ')
print('\n')
print("The node expansion order is ", end='')
for node in e:
    print(node.name, end = ' ')

The path returned is S B T G 

The node expansion order is S D A B J F T E J G 

# Sudoku Puzzle Problem

In [77]:
# constants for clearer code
FAILURE = False

N = 3 # N

# first create the variable list
vars = [var for var in range(N*N*N*N)]

# print(vars)

# create the domain for each variable in variables list
doms = {var:[pos + 1 for pos in range(N*N)]for var in vars} # this is a dictionary
# print(doms)
# # find the neighbors for each variable/ constraints

cols = []
rows = []
secs = []

for var in vars:
    col = []
    for v in vars:
        if var != v and v % 9 == var % 9:
            col.append(v)
    cols.append(col)

# print(cols)

for var in vars:
    row = []
    for v in vars:
        if var != v and v // 9 == var // 9:
            row.append(v)
    rows.append(row)

# print(rows)

for var in vars:
    sec = []
    for v in vars:
        if var != v:
            if var % 9 < 3 and var // 9 < 3 and v % 9 < 3 and v // 9 < 3: # top left section
                sec.append(v)
            elif var % 9 > 5 and var // 9 < 3 and v % 9 > 5 and v // 9 < 3: # top right section
                sec.append(v)
            elif var % 9 < 3 and var // 9 > 5 and v % 9 < 3 and v // 9 > 5: # bottom left section
                sec.append(v)
            elif var % 9 > 5 and var // 9 > 5 and v % 9 > 5 and v // 9 > 5: # bottom right section
                sec.append(v)
            elif 2 < var % 9 < 6 and var // 9 < 3 and 2 > v % 9 < 6 and v // 9 < 3: # top middle section
                sec.append(v)
            elif 2 < var % 9 < 6 and var // 9 > 5 and 2 < v % 9 < 6 and v // 9 > 5: # bottom middle section
                sec.append(v)
            elif var % 9 < 3 and 2 < var // 9 < 6 and v % 9 < 3 and 2 < v // 9 < 6: # left middle section
                sec.append(v)
            elif var % 9 > 5 and 2 < var // 9 < 6 and v % 9 > 5 and 2 < v // 9 < 6: # right middle section
                sec.append(v)
            elif 2 < var % 9 < 6 and 2 < var // 9 < 6 and 2 < v % 9 < 6 and 2 < v // 9 < 6: # center section
                sec.append(v)
    secs.append(sec)

# print(secs)

neighs = {var:[rows[var],cols[var],secs[var]] for var in vars}
            

# # create assigment list
assign = {var:[] for var in vars}  # create a dictionary with value as empty list because we have not assigned anything yet 

# # create a CSP problem dictionary
csp_2 = {"variables" : vars, "domains" : doms, "neighbors" : neighs}
csp_2


{'variables': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80],
 'domains': {0: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  1: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  2: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  3: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  4: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  5: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  6: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  7: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  8: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  9: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  10: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  11: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  12: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  13: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  14: [1, 2, 3, 4, 

In [78]:
csp_2["domains"][1] = [3]
csp_2["domains"][4] = [8]
csp_2["domains"][8] = [1]
csp_2["domains"][11] = [7]
csp_2["domains"][12] = [4]
csp_2["domains"][14] = [1]
csp_2["domains"][16] = [5]
csp_2["domains"][18] = [9]
csp_2["domains"][22] = [5]
csp_2["domains"][24] = [2]
csp_2["domains"][29] = [2]
csp_2["domains"][32] = [5]
csp_2["domains"][34] = [1]
csp_2["domains"][36] = [3]
csp_2["domains"][39] = [2]
csp_2["domains"][40] = [1]
csp_2["domains"][42] = [5]
csp_2["domains"][45] = [5]
csp_2["domains"][46] = [9]
csp_2["domains"][49] = [6]
csp_2["domains"][53] = [2]
csp_2["domains"][56] = [6]
csp_2["domains"][57] = [5]
csp_2["domains"][59] = [2]
csp_2["domains"][65] = [9]
csp_2["domains"][66] = [6]
csp_2["domains"][70] = [2]
csp_2["domains"][71] = [7]
csp_2["domains"][77] = [8]
csp_2["domains"][79] = [6]
csp_2["domains"][80] = [5]


csp_2

{'variables': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80],
 'domains': {0: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  1: [3],
  2: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  3: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  4: [8],
  5: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  6: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  7: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  8: [1],
  9: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  10: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  11: [7],
  12: [4],
  13: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  14: [1],
  15: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  16: [5],
  17: [1, 2, 3, 4, 5, 6, 7, 8, 9],
  18: [9],
  19: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 

In [86]:
import copy

def print_sudoku(assignment, CSP):
    """
    print the n queen
    """
    for row in CSP["variables"]:
        print(row, end="")
        for col in CSP["variables"]:
            if assignment[col][0] == row:
                print("| Q |", end="")
            else: 
                print("|   |", end="")
        print("\n")

def is_assignment_complete(assignment):
    """
    check if the assignment is complete. 
    E.g. 
        {0: [1], 1:[3], 2:[4]} is complete
        {0: [1], 1:[], 2:[4]}  is not complete as variable 1 has not been assigned
    """
    for k,v in assignment.items():
        if len(v) == 0: # check if the variable has been assigned yet
            return False
    return True

def select_unassigned_variable(assignment):
    """
    return 1 variable which has not been assigned yet
    E.g. 
        assignment = {0: [1], 1:[], 2:[4]} returns variable 1
    """
    for variable, assigned_value in assignment.items():
        if len(assigned_value) == 0: # check if the variable has been assigned yet
            return variable

def is_sudoku_consistent(value, variable, assignment, CSP):
    """
    return 1 variable which has not been assigned yet
    E.g. 
        assignment = {0: [1], 1:[], 2:[4]} returns variable 1
    """
    for k,v in assignment.items():
        if k != variable and len(v) != 0 and k in CSP["neighbors"][variable]:
            # first constraint: queens can't be on the same row
            if v[0] == value:
                return False
            # second constraint: queens can't see one another diagonally
            col_dist = abs(variable-k)
            if v[0] == value+col_dist or v[0] == value-col_dist:
                return False 
    return True

def forward_checking(CSP, variable, assignment):
    """ 
    Reduce the domain of neighbors of variable
    """
    # copy the dictionary to avoid pointing to the same dictionary
    new_CSP = copy.deepcopy(CSP)
    for var, value in assignment.items():
        if var != variable and var in new_CSP["neighbors"][variable]:
            if len(value) == 0: # if the variable has not been assigned yet
                if assignment[variable][0] in new_CSP["domains"][var]:
                    new_CSP["domains"][var].remove(assignment[variable][0])
            if len(new_CSP["domains"][var]) == 0:  # if the domain is reduced to empty
                return False, new_CSP
    return True, new_CSP

def none_inference(CSP, variable, assignment):
    return True, CSP

# implemement backtracking
def recursive_backtracking(assignment, CSP, check_consistent_function, inference=none_inference):
    if (is_assignment_complete(assignment) is True): 
        return True, assignment

    new_assignment = copy.deepcopy(assignment)
    variable = select_unassigned_variable(new_assignment)

    for value in CSP["domains"][variable]:
        if check_consistent_function(value, variable, new_assignment, CSP):
            # add the value to the new_assignment 
            new_assignment[variable].append(value)

            infererece_state, new_CSP = inference(CSP, variable, new_assignment)
            if infererece_state is True: # if the inference is successful
                backtrack_state, new_assignment = recursive_backtracking(new_assignment, new_CSP, check_consistent_function, inference)
                if backtrack_state is True:  
                    return True, new_assignment
                else:
                    new_assignment[variable].remove(value)
            else: # if the inference fails, i.e. some domains become empty
                new_assignment[variable].remove(value)

    return False, new_assignment


def revise_sudoku(CSP, Vi, Ni):
    revised = False
    new_CSP = copy.deepcopy(CSP)

    if len(CSP["domains"][Ni])==1:
        print("Domains: ")
        print(CSP["domains"][Vi])
        for vi in CSP["domains"][Vi]: # possible value for Vi
            
            ni = new_CSP["domains"][Ni][0] # value of neighbor

            if vi in new_CSP["domains"][Vi]: 
                if vi == ni: # possible value == only value for neighbor
                    new_CSP["domains"][Vi].remove(vi)
                    revised = True
    return revised, new_CSP


def arc_consistency(CSP, revise):
    queue = [(Vi, Ni) for Vi in CSP["variables"] for neighbors in CSP["neighbors"][Vi] for Ni in neighbors] 
    csp = copy.deepcopy(CSP)

    # for i in range(0,len(queue),8):
        # print(queue[i:i+8])
        

    while len(queue): # while queue is not empty
        Vi, Ni = queue.pop(0)
        print(f"Considering {Vi} -> {Ni}")
        revise_state, new_CSP = revise(csp, Vi, Ni)
        if revise_state is True: # the domain of Vi has been changed
            if len(new_CSP["domains"][Vi]) == 0:
                return False, new_CSP
            print(Vi + " Neighbors: ")
            print(new_CSP["neighbors"][Vi])
            for xk in new_CSP["neighbors"][Vi]:
                if xk != Ni:
                    queue.append((xk,Vi))
            csp = copy.deepcopy(new_CSP)

    return True, new_CSP



In [87]:
ac_state, new_csp = arc_consistency(csp_2, revise_sudoku)
new_csp

Considering 0 -> 1
Domains: 
[1, 2, 3, 4, 5, 6, 7, 8, 9]
Neighbors: 
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 18, 27, 36, 45, 54, 63, 72], [1, 2, 9, 10, 11, 18, 19, 20]]
Considering 0 -> 2
Considering 0 -> 3
Considering 0 -> 4
Domains: 
[1, 2, 4, 5, 6, 7, 8, 9]
Neighbors: 
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 18, 27, 36, 45, 54, 63, 72], [1, 2, 9, 10, 11, 18, 19, 20]]
Considering 0 -> 5
Considering 0 -> 6
Considering 0 -> 7
Considering 0 -> 8
Domains: 
[1, 2, 4, 5, 6, 7, 9]
Neighbors: 
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 18, 27, 36, 45, 54, 63, 72], [1, 2, 9, 10, 11, 18, 19, 20]]
Considering 0 -> 9
Considering 0 -> 18
Domains: 
[2, 4, 5, 6, 7, 9]
Neighbors: 
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 18, 27, 36, 45, 54, 63, 72], [1, 2, 9, 10, 11, 18, 19, 20]]
Considering 0 -> 27
Considering 0 -> 36
Domains: 
[2, 4, 5, 6, 7]
Considering 0 -> 45
Domains: 
[2, 4, 5, 6, 7]
Neighbors: 
[[1, 2, 3, 4, 5, 6, 7, 8], [9, 18, 27, 36, 45, 54, 63, 72], [1, 2, 9, 10, 11, 18, 19, 20]]
Considering 0 -> 54
Considering 0 -> 63
Consider

{'variables': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80],
 'domains': {0: [2, 4, 6],
  1: [3],
  2: [4, 5],
  3: [7],
  4: [8],
  5: [4, 6],
  6: [4, 6, 9],
  7: [4, 9],
  8: [1],
  9: [2, 6, 8],
  10: [2, 6, 8],
  11: [7],
  12: [4],
  13: [2],
  14: [1],
  15: [3, 6, 8, 9],
  16: [5],
  17: [3, 6, 8, 9],
  18: [9],
  19: [1, 4, 6, 8],
  20: [1, 4, 8],
  21: [1, 8],
  22: [5],
  23: [4, 6, 7],
  24: [2],
  25: [3, 4, 7, 8],
  26: [3, 4, 6, 8],
  27: [4, 6, 7, 8],
  28: [4, 6, 7, 8],
  29: [2],
  30: [3, 8, 9],
  31: [3, 4, 7, 9],
  32: [5],
  33: [3, 4, 6, 7, 8

In [ ]:
state, ass = recursive_backtracking(assign, csp_2, is_sudoku_consistent)

In [ ]:
state

In [ ]:
print_sudoku(ass, new_csp)

In [ ]:
state, ass = recursive_backtracking(assign, csp_2, is_sudoku_consistent, forward_checking)

In [ ]:
print_sudoku(ass, csp_2)